# Walkthrough of 10-K Data Extraction
Here we take a look at the classes and functions that are used in the `main.py` script to generate a folder of `.csv` files for each statement-type table in a 10-K, for each year that has an xml summary.

We will produce a single `.csv` file at the end of this notebook.

In [1]:
# Packages we will need:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
from urllib.request import urlopen
import time

### Create first class.
`JsonLinks` provides us with `index.json` links we can find 10-K tables with.

In [2]:
class JsonLinks:
    """LinkEndings class takes a CIK and form-type (defaults to 10-K)
    and returns a list of links that can (possibly) be used to extract 
    10-K tables and write them as CSVs or push them to a database.
    """
    def __init__(self, cik, formType='10-K'):
        self.cik = cik
        self.fullCik = self._add_zeros_to_cik()
        self.formType = formType
        

    def _add_zeros_to_cik(self):
        """Class method to ensure CIK is of the correct length.  This
        method also ensures we will be working with a CIK of a standard
        length, i.e. 10 characters.
        The method is not strictly necessary for what we are doing.
        """
        L = len(self.cik)
        if L == 1:
            self.cik = '000000000' + self.cik
            return self.cik
        elif L == 2:
            self.cik = '00000000' + self.cik
            return self.cik
        elif L == 3:
            self.cik = '0000000' + self.cik
            return self.cik
        elif L == 4:
            self.cik = '000000' + self.cik
            return self.cik
        elif L == 5:
            self.cik = '00000' + self.cik
            return self.cik
        elif L == 6:
            self.cik = '0000' + self.cik
            return self.cik
        elif L == 7:
            self.cik = '000' + self.cik
            return self.cik
        elif L == 8:
            self.cik = '00' + self.cik
            return self.cik
        elif L == 9:
            self.cik = '0' + self.cik
            return self.cik
        elif L == 10:
            self.cik = self.cik
            return self.cik
        else:
            print('\n***Error encountered: invalid CIK.***\n')
    
    
    def build_cik_search_url(self):
        """Create the url we need to use to check if a CIK has any 10-K
        reports listed.  From these 10-Ks, we will eventually figure 
        out if any of them have .xml summaries.
        """
        url0 = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK='
        url1 = '&type=' 
        url2 = '&dateb=&owner=include&count=100&search_text='
        url = url0 + self.fullCik + url1 + self.formType + url2
        return url
    

    def get_soup(self):
        """Get the BeautifulSoup of the url we built in the
        build_cik_search_url().
        """
        urlClient = urlopen(self.build_cik_search_url())
        time.sleep(0.2)
        webPage = urlClient.read()
        urlClient.close()
        soupData = BeautifulSoup(webPage, 'html.parser')
        return soupData
    
    
    def get_10k_a_tags(self):
        """Find all <a> tags within the 10-K BeautifulSoup that have an
        id of 'documentsbutton'.
        """
        aTags = self.get_soup().find_all('a', id='documentsbutton')
        return aTags
    
    
    def get_10k_link_endings(self):
        """Create a list of all the links within the <a> tags found in 
        get_10k_a_tags().
        """
        linkEndings = []
        for aTag in self.get_10k_a_tags():
            linkEndings.append(aTag.get('href'))
        return linkEndings
    
    
    def add_base_url(self):
        """Patch together the SEC-portion of the link and the link 
        endings from get_10k_link_endings().
        """
        links = []
        baseUrl = r'https://www.sec.gov'
        for link in self.get_10k_link_endings():
            link = baseUrl + link
            links.append(link)
        return links
    
    
    def _get_htm_links(self):
        """Class-method to make a list of all .htm links."""
        htmLinks = []
        for link in self.add_base_url():
            if link.endswith('.htm'):
                htmLinks.append(link)
            else:
                pass
        return htmLinks   


    def get_json_htm_links(self):
        """All the .htm 10-K links are turned into links ending with 
        .json instead.  This involves removing the CIK, the date, and
        the incremental number of the document for that year, along 
        with removing 'index.htm'.  We parse through the .json to look 
        for .xml summaries.  The .xml summaries are what provide us 
        with all the links to a 10-K's tables.
        """
        jsonLinks = []
        for link in self._get_htm_links():
            # Reverse the link to access part we delete, in consistent manner.
            link = link[::-1]
            # Remove CIK, date, and document filing number. 
            link = link.replace(link[9:30], '')
            # Flip link back to way it was, this time without -numbers-
            link = link[::-1] 
            # Replace the index.htm ending with the index.json ending.
            link = link.replace('index.htm', 'index.json')
            # Add the formatted link to our list of json links.
            jsonLinks.append(link)
        return jsonLinks

Take the `JsonLinks` class for a spin.

In [3]:
cik = '2488' # AMD's CIK.
J = JsonLinks(cik)
cik = J.fullCik
print("CIK with added zeros:", cik) # Check adding leading zeros works.
jsonUrls = J.get_json_htm_links()
print("\nList of jsonUrls we can try in the next stage:\n", jsonUrls) # Print out list of jsonUrls we created.

CIK with added zeros: 0000002488

List of jsonUrls we can try in the next stage:
 ['https://www.sec.gov/Archives/edgar/data/2488/000162828021001185/index.json', 'https://www.sec.gov/Archives/edgar/data/2488/000000248820000008/index.json', 'https://www.sec.gov/Archives/edgar/data/2488/000000248819000011/index.json', 'https://www.sec.gov/Archives/edgar/data/2488/000000248818000042/index.json', 'https://www.sec.gov/Archives/edgar/data/2488/000000248817000043/index.json', 'https://www.sec.gov/Archives/edgar/data/2488/000000248816000111/index.json', 'https://www.sec.gov/Archives/edgar/data/2488/000119312515054362/index.json', 'https://www.sec.gov/Archives/edgar/data/2488/000119312514057240/index.json', 'https://www.sec.gov/Archives/edgar/data/2488/000119312513069422/index.json', 'https://www.sec.gov/Archives/edgar/data/2488/000119312512075837/index.json', 'https://www.sec.gov/Archives/edgar/data/2488/000119312511040392/index.json', 'https://www.sec.gov/Archives/edgar/data/2488/0001193125100

To see what links look like under the hood, access functions within `JsonLinks`.
Uncomment or change line 1 in the cell below.

In [4]:
#print(J.get_10k_link_endings())

### Create second class.
`StatementUrls` gives us a list of the urls for tables that are in a 10-K.

In [5]:
class StatementUrls:
    """StatementUrls takes an <index.json> url (created with the 
    JsonLinks class) and produces a list of table urls (R[0-100].htm)
    that can then be used to extract the data from financial statements 
    on EDGAR.
    """
    def __init__(self,url):
        self.url = url
        self.baseUrl = 'https://www.sec.gov'

    
    def get_xml_link(self):
        """If an index.json link has a FilingSummary.xml, make it into
        a usable link, otherwise pass.
        """
        content = requests.get(self.url).json()
        for file in content['directory']['item']:
            if file['name'] == 'FilingSummary.xml':
                xmlSummary = (
                    self.baseUrl 
                    + content['directory']['name'] 
                    + '/' 
                    + file['name']
                )
            else:
                pass
        return xmlSummary
        

    def get_master_reports(self):
        """Make a list of dictionaries that include the name, category and 
        url of every table in the 10-K.
        """
        baseUrl = self.get_xml_link().replace('FilingSummary.xml', '')

        content = requests.get(self.get_xml_link()).content
        soupData = BeautifulSoup(content, 'lxml')
        reports = soupData.find('myreports') # Xml tag enclosing all table reports.
        
        masterReports = [] # Create list to hold dictionaries of report tables.

        # Avoid last one (causes error)
        for report in reports.find_all('report')[:-1]: 
            # Create dictionary to store the parts we need
            reportDictionary = {}
            reportDictionary['nameShort'] = report.shortname.text
            reportDictionary['nameLong'] = report.longname.text
            reportDictionary['position'] = report.position.text
            reportDictionary['category'] = report.menucategory.text
            reportDictionary['url'] = baseUrl + report.htmlfilename.text    
            # Append the dictionary to the master list
            masterReports.append(reportDictionary)           
        return masterReports
    
    
    def get_statement_reports(self):
        """Pull out all dictionaries that have
            'category': 'Statements'

        Note that there are usually 50-100 items in these reports, with
        the vast majority containing superfluous information.  Thus, we
        are only pulling 'statements' from the reports in the current
        configuration.
        """
        statementsDictionary = []
        for reportDictionary in self.get_master_reports():
            if reportDictionary['category'].lower().find('statements') != -1:
                print('-'*100)
                print(reportDictionary['nameShort'])
                print(reportDictionary['url'])
                statementsDictionary.append(reportDictionary)           
            else:
                pass
        return statementsDictionary
    
    
    def get_statement_urls(self):
        statementUrls = []
        for statement in self.get_statement_reports():
            statementUrls.append(statement['url'])
        return statementUrls

In [6]:
# Test using the most recent .json url. Ones older than 5-6 years don't
#  seem to have xml summaries, which is what ensures there are links
#  to each table in a 10-K.
testUrl = jsonUrls[0]
print(testUrl)

https://www.sec.gov/Archives/edgar/data/2488/000162828021001185/index.json


If the link above leads to a page of json that has `"name":"R[1-100].htm"` throughout, we are in good shape. These are the endings for links to tables in a 10-K. 

If inclined, see what the link shows for `testUrl = jsonUrls[-1]`. From this url, we can create a link like [this](https://www.sec.gov/Archives/edgar/data/2488/000101287000004907/0001012870-00-004907.txt)
(`https://www.sec.gov/Archives/edgar/data/2488/000101287000004907/0001012870-00-004907.txt`):
which is how table data from an older (circa 2000) 10-K is presented. This older format of 10-K data is beyond the scope of what this code scrapes and formats, but it could be an interesting problem to work on.  

In [7]:
# Create the statement urls for our testUrl.
# We should see links for the 'Statements' tables and their names.
statementUrls = StatementUrls(testUrl).get_statement_urls()

----------------------------------------------------------------------------------------------------
Consolidated Statements of Operations
https://www.sec.gov/Archives/edgar/data/2488/000162828021001185/R2.htm
----------------------------------------------------------------------------------------------------
Consolidated Statements of Comprehensive Income
https://www.sec.gov/Archives/edgar/data/2488/000162828021001185/R3.htm
----------------------------------------------------------------------------------------------------
Consolidated Balance Sheets
https://www.sec.gov/Archives/edgar/data/2488/000162828021001185/R4.htm
----------------------------------------------------------------------------------------------------
Consolidated Balance Sheets (Parenthetical)
https://www.sec.gov/Archives/edgar/data/2488/000162828021001185/R5.htm
----------------------------------------------------------------------------------------------------
Consolidated Statements of Stockholders Equity
https:

Note by changing code on line 67 of `StatementUrls`, we can create the links for every table within a 10-K.
Our code only pulls tables from the `/FilingSummary.xml` that have `'category': 'Statements'` in the json.

If you want to see the filing summary uncomment line 1 below (and follow link):
`print(StatementUrls(testUrl).get_xml_link())`.

If you want to see the json of the filing summary, uncomment line 2 below:
`print(StatementUrls(testUrl).get_master_reports())`.

In [8]:
#print(StatementUrls(testUrl).get_xml_link())
#print(StatementUrls(testUrl).get_master_reports())

### Define function to make dictionaries of a table's data.

In [9]:
def create_statement_data(statementUrl):
    """Once we have valid table_urls from our StatementUrls function, 
    we pass those urls into this function to yield a list of 
    dictionaries that separate the table's contents by headers, 
    sections, and data.

    `statementUrl`: LIST
    A list of urls that link to statement-type tables in a 10-K.
    """
    # Assume we want all statements in a single data list.
    statementsData = []

    # Loop through each statementUrl
    for statement in statementUrl:
    
        # Define dictionary that will store the different parts of the statement.
        statementData = {}
        statementData['headers'] = []
        statementData['sections'] = []
        statementData['data'] = []
    
        # Request the statement file content
        content = requests.get(statement).content
        reportSoup = BeautifulSoup(content, 'lxml')
    
        """Find all the rows, figure out what type of row, parse elements
        and store in the statement file list.
        """
        for index, row in enumerate(reportSoup.table.find_all('tr')):
        
            # Get all the elements
            columns = row.find_all('td')
        
            # If regular row and not a section or a table header.
            if (len(row.find_all('th')) == 0 and len(row.find_all('strong')) == 0):
                regularRow = [ele.text.strip() for ele in columns]
                statementData['data'].append(regularRow)
            
            # If regular row and section but not a table header.
            elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
                sectionRow = columns[0].text.strip()
                statementData['sections'].append(sectionRow)
            
            # If not either of those, it must be header.
            elif (len(row.find_all('th')) != 0):
                headRow = [ele.text.strip() for ele in row.find_all('th')]
                statementData['headers'].append(headRow)
            
            else:
                print('We encountered an error making a statement dictionary.')
            
        # Append it to the master list, statementsData.
        statementsData.append(statementData)
    
    return statementsData

In [10]:
statementData = create_statement_data(statementUrls)

In [11]:
# statementData is our list of dictionaries that correspond
#  to each link we have for a 10-K's `statement` table.
statementData

[{'headers': [['Consolidated Statements of Operations - USD ($) shares in Millions, $ in Millions',
    '12 Months Ended'],
   ['Dec. 26, 2020', 'Dec. 28, 2019', 'Dec. 29, 2018']],
  'sections': ['Income Statement [Abstract]',
   'Earnings per share',
   'Shares used in per share calculation'],
  'data': [['Net revenue', '$ 9,763', '$ 6,731', '$ 6,475'],
   ['Cost of sales', '5,416', '3,863', '4,028'],
   ['Gross profit', '4,347', '2,868', '2,447'],
   ['Research and development', '1,983', '1,547', '1,434'],
   ['Marketing, general and administrative', '995', '750', '562'],
   ['Operating income', '1,369', '631', '451'],
   ['Interest expense', '(47)', '(94)', '(121)'],
   ['Other expense, net', '(47)', '(165)', '0'],
   ['Income before income taxes and equity income (loss)',
    '1,275',
    '372',
    '330'],
   ['Income tax provision (benefit)', '(1,210)', '31', '(9)'],
   ['Equity income (loss) in investee', '5', '0', '(2)'],
   ['Net income', '$ 2,490', '$ 341', '$ 337'],
   ['Bas

### Define function to transform list of dictionaries into a DataFrame.

In [14]:
def create_statement_dataframe(statementsData, statementNumber=0):
    """Once we have a list of dictionaries representing all the 10-K 
    statement-type tables (built by create_statement_data()), we pass
    the list to this function.  This function attempts to transform a
    given dictionary into a dataframe that can then be easily 
    manipulated. 
    
    `statementsData`: LIST (of dictionaries).
    The output of create_statement_data(), a list
    of dictionaries for the statement-type tables in a 10-K.

    `statementNumber`: INT
    Number of the dictionary from the list that
    holds the statement-type table urls.
    """
    
    # Grab the proper components
    statementHeader =  statementsData[statementNumber]['headers']
    data = statementsData[statementNumber]['data']

    
    ### Logic to make sure correct headers are being used.
    print('Length of header list:', len(statementHeader) )
    # Get title of the dataframe index
    title = statementsData[statementNumber]['headers'][0][0]
    print('Index title:', statementsData[statementNumber]['headers'][0][0])

    if len(statementHeader) == 2:
        statementHeader =  statementsData[statementNumber]['headers'][1]
        print('Column headers that will be used:')
        print(statementHeader)
        widthStatementHeader = len(statementHeader)
        print('Width of header is:', widthStatementHeader)
    elif len(statementHeader) == 1:
        statementHeader =  statementsData[statementNumber]['headers'][0][1:]
        print('Column headers that will be used:')
        print(statementHeader)
        widthStatementHeader = len(statementHeader)
        print('Width of header is:', widthStatementHeader)
    else:
        print('Length of statementHeader is:', len(statementHeader))
        print('Need more case logic.')
        pass
    ### End logic.
    
    
    ### Logic to remove rows that are of a different length than the first one.
    dfWidth = len(data[0])
    print('Width of dataframe is:', dfWidth, '\n')
    numRows = len(data[:])
    newData = []
    for i in range(numRows):
        if dfWidth == len(data[i]):
            print('Keeping this row: ', data[i])
            newData.append(data[i])
        else: 
            print('Removing this row: ', data[i])
            pass
    print('\nNumber of rows that were in dataframe:', numRows)
    print('Number of rows in dataframe now:', len(newData[:]))
    df = pd.DataFrame(newData)
    ### End logic.

    # Define the Index column, rename it, and we need to make sure to 
    #  drop the old column once we reindex.
    df.index = df[0]
    # Replace USD symbol with the little one so dataframe will display 
    #  it without thinking it's a LaTeX expression.
    title = title.replace('$', '﹩')
    df.index.name = title
    df = df.drop(0, axis=1)

    # Get rid of the '$', '(', ')', '%', and convert the '' to np.NaNs.
    df = df.replace('[\$,)]','', regex=True )\
           .replace( '[(]','-', regex=True)\
           .replace('\[[0-9]\]','', regex=True)\
           .replace( '', np.NaN, regex=True)\
           .replace('\%','', regex=True)\

    ### Logic to remove column of all NaN values.
    """If the number of headers + 1 is equal to the number of columns in the dataframe,
    that's what we want:
    """
    if dfWidth == (widthStatementHeader + 1):
        pass
    elif dfWidth >= (widthStatementHeader + 2):
        """Look for columns with all NaN values to delete, since 
        that's most likely the problem.
        """
        print('Finding NaN column(s) to remove')
        df = df.dropna(axis=1, how='all')
    else:
        print('Columns of dataframe:', dfWidth)
        print('Columns of headers:', widthStatementHeader)
        print('The difference is either greater than two or less than one.')
        print('Please add more logic to create_statement_dataframe().')
        pass
    ### Logic end.

    """Everything is a string, let's convert all the data to a float.
    Specifying error='ignore' will churn out the table with headers still intact.
    errors='raise' (default) will cause a table failure if data not a float.
    """
    df = df.astype(float, errors='ignore') 
    df.columns = statementHeader
    return df

In [15]:
# Create the dataframe of a statement.
# It defaults to 0 in the funciton.
# Enter any other number within the range of tables we made.
tableNumber = 0 # Used again in the name of the .csv file we make.
df = create_statement_dataframe(statementData, tableNumber)

Length of header list: 2
Index title: Consolidated Statements of Operations - USD ($) shares in Millions, $ in Millions
Column headers that will be used:
['Dec. 26, 2020', 'Dec. 28, 2019', 'Dec. 29, 2018']
Width of header is: 3
Width of dataframe is: 4 

Keeping this row:  ['Net revenue', '$ 9,763', '$ 6,731', '$ 6,475']
Keeping this row:  ['Cost of sales', '5,416', '3,863', '4,028']
Keeping this row:  ['Gross profit', '4,347', '2,868', '2,447']
Keeping this row:  ['Research and development', '1,983', '1,547', '1,434']
Keeping this row:  ['Marketing, general and administrative', '995', '750', '562']
Keeping this row:  ['Operating income', '1,369', '631', '451']
Keeping this row:  ['Interest expense', '(47)', '(94)', '(121)']
Keeping this row:  ['Other expense, net', '(47)', '(165)', '0']
Keeping this row:  ['Income before income taxes and equity income (loss)', '1,275', '372', '330']
Keeping this row:  ['Income tax provision (benefit)', '(1,210)', '31', '(9)']
Keeping this row:  ['Equi

In [16]:
# Look at the dataframe we created for the 10-K statement table.
display(df)

,"Dec. 26, 2020","Dec. 28, 2019","Dec. 29, 2018"
"Consolidated Statements of Operations - USD (﹩) shares in Millions, ﹩ in Millions",,,
Net revenue,9763.00,6731.00,6475.00
Cost of sales,5416.00,3863.00,4028.00
Gross profit,4347.00,2868.00,2447.00
Research and development,1983.00,1547.00,1434.00
"Marketing, general and administrative",995.00,750.00,562.00
Operating income,1369.00,631.00,451.00
Interest expense,-47.00,-94.00,-121.00
"Other expense, net",-47.00,-165.00,0.00
Income before income taxes and equity income (loss),1275.00,372.00,330.00


In [17]:
# Define function for extracting the year of the 10-K report.
def get_10k_year_from_url(url):
    """From a link like this,
    <https://www.sec.gov/Archives/edgar/data/2488/000162828021001185/index.json>
    the function flips the url around (for consistency between links) 
    and takes off the first 17 characters <001185/index.json>.  Then it
    flips the url back around and takes the last two characters <21>.  
    Finally, it adds '20' to the <21> in order to return '2021', as 
    desired.
    """
    url = url[::-1] # Reverse the url.
    # Remove everything up to the two digits of the year
    url = url.replace(url[0:17], '') 
    url = url[::-1] # Reverse the url back.
    twoDigitYear = url[-2:]
    cikYear = '20' + twoDigitYear
    return cikYear

Finally, we can save our dataframe as a `.csv` in our local `./data/` folder.

In [18]:
year10k = get_10k_year_from_url(testUrl)
tableName = cik + "_" + year10k + "_table" + str(tableNumber)
pathOut = './data/'
df.to_csv(pathOut + tableName + ".csv")